In [1]:
import sys
sys.path.append('..')
from api.api import PDTB
import re
from ete3 import Tree
from collections import Counter
from enum import Enum

In [21]:
# pdtb2 = PDTB('/home/pengfei/data/pdtb_v2/all/conll/')
pdtb3 = PDTB('/home/pengfei/data/pdtb3/all/conll/')

In [3]:
i=15
pdtb3.get_simplified_parsetree_string(i)

['(S )', '(S )']

In [11]:
syntax_trees, _ = pdtb3.get_highlighted_parsetree_with_word(15)

In [7]:
pt = ProcessTree()

In [8]:
class ProcessTree:
    def __init__(self):
        pass
    
    def get_simplified_tree_string(self, tree):
        tree_name = self.simplify_tree_with_word(tree)
        tree = self.more_simplify_tree_with_word(tree)
        tree = self.most_simplify_tree_with_word(tree)
        tree_string = self.tree2string(tree)
        sentence = ' '.join([o.split()[-1] for o in re.findall(r'\((.+?)\)', tree_string)])
        return tree_string, sentence
    
    
    def simplify_tree_with_word(self, tree):
        temp = []
        tag = tree.name.split()[0]
        if tag in ['Arg1', 'Arg2', 'none', 'Conn']:
            return tree.name
        for c in tree.children:
            child_name = self.simplify_tree_with_word(c)
            temp.append(child_name.split()[0])
        if len(set(temp)) == 1:
            tree.name = temp[0] + ' ' + ' '.join([c.name[4:] for c in tree.children])
            tree.children = []
        return tree.name


    def more_simplify_tree_with_word(self, tree):
        temp = []
        # simplify among siblings
        prev_c = None
        for c in tree.children:
            c = self.more_simplify_tree_with_word(c)
            if prev_c == None:
                temp.append(c)
                prev_c = c
                continue
            if c.name.split()[0] != prev_c.name.split()[0]:
                temp.append(c)
                prev_c = c
            else:
                prev_c.name = prev_c.name + c.name[4:]
        tree.children = temp
        return tree

    def most_simplify_tree_with_word(self, tree):
        for i, c in enumerate(tree.children):
            tree.children[i] = self.most_simplify_tree_with_word(c)
        if len(tree.children) == 2:
            # for left branch
            if len(tree.children[1].children) >= 1:
                if tree.children[1].children[0].name.split()[0] == tree.children[0].name.split()[0]:
                    downup = tree.children[1].children[0]
                    downup.name = downup.name.split()[0] + tree.children[0].name[4:] + downup.name[4:]
                    tree = tree.children[1]
            # for right branch
            if len(tree.children[0].children) >= 1:
                if tree.children[0].children[1].name.split()[0] == tree.children[1].name.split()[0]:
                    updown = tree.children[0].children[1]
                    updown.name = updown.name + tree.children[1].name[4:]
                    tree = tree.children[0]
        tree = self.more_simplify_tree_with_word(tree)
        return tree


    def tree2string(self, tree):
        tree_string = ""
        if len(tree.children) == 0:
            return ''.join(["(" + tree.name.split()[0] + " " + o + ")" for o in tree.name.split()[1:]])
        tree_string += "(" + tree.name + " "
        for c in tree.children:
            tree_string += self.tree2string(c)
        tree_string += ")"
        return tree_string

In [22]:
ret = []
pdtb3.parse_data = pdtb3.parse_data_test
for i in range(len(pdtb3.parse_data)):
    if pdtb3.get_arg_sent_id(i, 'Arg1') == pdtb3.get_arg_sent_id(i, 'Arg2') and len(pdtb3.get_arg_sent_id(i, 'Arg1')) == 1:
        syntax_trees, _ = pdtb3.get_highlighted_parsetree_with_word(i)
        for s in syntax_trees:
            tree_string, string = pt.get_simplified_tree_string(syntax_trees[0].tree)
            ret.append(tree_string)

Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not generated
Parse tree is not ge

In [23]:
def write(filename, folder):
    with open(filename, 'w') as f:
        for o in folder:
            f.write(o + '\n')

write('test.txt', ret)

## TODO:
1. the label must be arg1 arg2 conn or none then able to merge, if the label is constituent tree label. do not merge.
2. what if parent have label and become child in most simplified tree function